# SDFGJKL

In [20]:
import math
 #This code represents the conversion of a simple window model into a more equivalent complex feature window
def calculate_glass_properties(U, SHGC, tvis, epsilon=0.84, tau_longwave=0):
    # Step 1: Calculate overall thermal resistance Rl,w
    if U < 5.85:
        Ri_w = 1 / (0.359073 * math.log(U) + 6.949915)
    else:
        Ri_w = 1 / (1.788041 * U - 2.886625)
        
    Ro_w = 1 / (0.025342 * U + 29.163853)
    Rl_w = 1 / U - Ri_w - Ro_w

    print(f"Step 1 - Ri_w: {Ri_w}, Ro_w: {Ro_w}, Rl_w: {Rl_w}")

    # Step 2: Calculate glass thickness and conductivity k
    if 1 / Rl_w > 7:
        thickness = 0.002
    else:
        thickness = 0.05914 - 0.00714 / Rl_w

    k = thickness / Rl_w

    print(f"Step 2 - thickness: {thickness}, k: {k}")

    # Step 3: Calculate Tsol based on U and SHGC
    if U > 4.5:
        if SHGC < 0.7206:
            Tsol = 0.939998 * SHGC ** 2 + 0.20332 * SHGC
        else:
            Tsol = 1.30415 * SHGC - 0.30515
    elif U < 3.4:
        if SHGC <= 0.15:
            Tsol = 0.4104 * SHGC
        else:
            Tsol = 0.085775 * SHGC ** 2 + 0.963954 * SHGC - 0.084958
    else:
        if SHGC <= 0.15:
            Tsol_3_4 = 0.4104 * SHGC
            Tsol_4_5 = 0.939998 * SHGC ** 2 + 0.20332 * SHGC
        elif SHGC <= 0.7206:
            Tsol_3_4 = 0.085775 * SHGC ** 2 + 0.963954 * SHGC - 0.084958
            Tsol_4_5 = 0.939998 * SHGC ** 2 + 0.20332 * SHGC
        else:
            Tsol_3_4 = 0.085775 * SHGC ** 2 + 0.963954 * SHGC - 0.084958
            Tsol_4_5 = 1.30415 * SHGC - 0.30515
        
        Tsol = (Tsol_4_5 - Tsol_3_4) * (U - 3.4) / (4.5 - 3.4) + Tsol_3_4

    print(f"Step 3 - Tsol: {Tsol}")

    # Step 4: Calculate Rs,f and Rs,b
    if U >= 4.5:
        Ri_s = 1 / (29.436546 * (SHGC - Tsol) ** 3 - 21.943415 * (SHGC - Tsol) ** 2 + 9.945872 * (SHGC - Tsol)+7.426151)
        Ro_s = 1 / (2.225824 * (SHGC - Tsol) + 20.57708)
    elif U <= 3.4:
        Ri_s = 1 / (199.8208128 * (SHGC - Tsol) ** 3 - 90.639733 * (SHGC - Tsol) ** 2 + 19.737055 * (SHGC - Tsol) + 6.766575)
        Ro_s = 1 / (5.763355 * (SHGC - Tsol) + 20.541528)
    else:
        Ri_s_3_4 = 1 / (199.8208128 * (SHGC - Tsol) ** 3 - 90.639733 * (SHGC - Tsol) ** 2 + 19.737055 * (SHGC - Tsol) + 6.766575)
        Ro_s_3_4 = 1 / (5.763355 * (SHGC - Tsol) + 20.541528)
        Ri_s_4_5 = 1 / (29.436546 * (SHGC - Tsol) ** 3 - 21.943415 * (SHGC - Tsol) ** 2 + 9.945872 * (SHGC - Tsol)+7.426151)
        Ro_s_4_5 = 1 / (2.225824 * (SHGC - Tsol) + 20.57708)
        Ri_s = (Ri_s_4_5 - Ri_s_3_4) * (U - 3.4) / (4.5 - 3.4) + Ri_s_3_4
        Ro_s = (Ro_s_4_5 - Ro_s_3_4) * (U - 3.4) / (4.5 - 3.4) + Ro_s_3_4

    print(f"Step 4 - Ri_s: {Ri_s}, Ro_s: {Ro_s}")

    Fac = (Ro_s + 0.5 * Rl_w) / (Ro_s + Rl_w + Ri_s)
    Rs_f = Rs_b = 1 - Tsol - (SHGC - Tsol) / Fac

    print(f"Step 4 - Fac: {Fac}, Rs_f: {Rs_f}, Rs_b: {Rs_b}")

    # Step 5: Calculate visible light reflectance
    Rvis_b = -0.7409 * tvis ** 3 + 1.6531 * tvis ** 2 - 1.2299 * tvis + 0.4547
    Rvis_f = -0.0622 * tvis ** 3 + 0.4277 * tvis ** 2 - 0.4169 * tvis + 0.2399

    print(f"Step 5 - Rvis_f: {Rvis_f}, Rvis_b: {Rvis_b}")

    # Longwave emissivity and transmittance
    longwave_emissivity = epsilon
    longwave_transmittance = tau_longwave

    return {
        "thickness": thickness,
        "k": k,
        "Tsol": Tsol,
        "Rs_f": Rs_f,
        "Rs_b": Rs_b,
        "Rvis_f": Rvis_f,
        "Rvis_b": Rvis_b,
        "longwave_emissivity": longwave_emissivity,
        "longwave_transmittance": longwave_transmittance
    }

# Example usage
U = 3.69
SHGC = 0.717
tvis = 0.774

properties = calculate_glass_properties(U, SHGC, tvis)
print(properties)


Step 1 - Ri_w: 0.13479395689084833, Ro_w: 0.03417942800671177, Rl_w: 0.1020293251295402
Step 2 - thickness: 0.002, k: 0.01960220747771022
Step 3 - Tsol: 0.6446854749346637
Step 4 - Ri_s: 0.12724440758865102, Ro_s: 0.04784739342407256
Step 4 - Fac: 0.3567467315288352, Rs_f: 0.15260902372708157, Rs_b: 0.15260902372708157
Step 5 - Rvis_f: 0.14460300914720003, Rvis_b: 0.1495458494984001
{'thickness': 0.002, 'k': 0.01960220747771022, 'Tsol': 0.6446854749346637, 'Rs_f': 0.15260902372708157, 'Rs_b': 0.15260902372708157, 'Rvis_f': 0.14460300914720003, 'Rvis_b': 0.1495458494984001, 'longwave_emissivity': 0.84, 'longwave_transmittance': 0}
